# Economic Data Processing and Analysis

## Overview
This notebook processes economic data from multiple Excel files, handles missing data, and prepares the dataset for analysis. The data includes various economic indicators such as GDP, debt ratios, inflation, and unemployment rates.

## Data Sources
- **IMF Data**: GDP, inflation, unemployment, government debt metrics
- **BIS Data**: Debt service ratios for households and corporations

## Processing Steps
1. [Environment Setup - Data Loading](#env-setup)  
2. [Data Cleaning - Missing Values](#data-cleaning)   
3. [Time Period Analysis](#time-analysis)
4. [Country Selection Analysis](#country-analysis) 
5. [Data Imputation and Final Cleaning](#data-imputation) 
6. [BIS Data Integration](#bis-integration) 
7. [Data Format Transformation](#data-format)

<a id="env-setup"></a>
## 1. Environment Setup - Data Loading
### Required Libraries
Install the necessary libraries before running this notebook:

```bash
pip install pandas numpy matplotlib openpyxl
```

In [ ]:
# Import required libraries
import pandas as pd
import glob
import numpy as np 
import matplotlib.pyplot as plt

# Set display options for better readability
pd.set_option('display.max_columns', None)
pd.set_option('display.width', None)
pd.set_option('display.max_colwidth', None)

### File Discovery and Filtering
We load all Excel files from the Data folder, excluding BIS debt service ratio files which require special handling due to their different structure.


In [ ]:
# Define files to handle separately (BIS data has different structure)
ignore_list=['Data\\Debt service ratios - Households (BIS).xlsx','Data\\Debt service ratios - Nonfinancial corporate (BIS).xlsx']
# Get all Excel files from Data folder
files = glob.glob("Data/*.xlsx") 
files.remove(ignore_list[0])   
files.remove(ignore_list[1])
print(files)
print('Number of files in Data folder: '+str(len(files)))

### Loading Data into DataFrames
Each Excel file is loaded into a separate DataFrame and stored in a list for batch processing.


In [ ]:
# Initialize list to store DataFrames
df_list=[]
for i,file in enumerate(files,1):
    # Extract clean filename
    name=file.split('\\')[-1]
    name=name.replace('.xlsx', '') 
    print(f'Loading {i}/{len(files)}: {name}')
    
    df_name = pd.read_excel(file, engine="openpyxl")
    df_list.append(df_name)
    # Display basic info about the dataset

    print(f"   Shape: {df_name.shape}")
    print(f"   Columns: {list(df_name.columns[:5])}{'...' if len(df_name.columns) > 5 else ''}")
    print('    Sample data:' )
    print(df_name.head(2).to_string())
    print("- " * 80)



In [ ]:
# Calculate total data points
total_rows = sum(len(df) for df in df_list)
print("Total data points (rows):", total_rows)

total_cells = sum(df.size for df in df_list)
print("Total data points (cells):", total_cells)



<a id="data-cleaning"></a>
## 2. Data Cleaning - Missing Values
### Standardizing Missing Data Representation
Convert 'no data' strings to NaN for consistent handling of missing values.


In [ ]:
for i, df in enumerate(df_list):
    df_list[i] = df.replace('no data', np.nan,regex=False)


### Analyzing Missing Data Patterns
Calculate the percentage of missing data for each variable to understand data quality.


In [ ]:
missing_list=[]

for i,df_metric in enumerate(df_list, 1):
    missing_pct = df_metric.isnull().mean() * 100
    
    missing_list.append(missing_pct)
    print(f"Sample Data: \n --{missing_list[-1].head(4)}")
    print(f"Total missing values: {df_metric.isnull().sum().sum()}")
    print(f"Average missing percentage: {missing_pct.mean():.2f}%")
    
 

<a id="time-analysis"></a>
## 3. Time Period Analysis

### Creating Missing Data by Year Matrix
We create a comprehensive view of missing data patterns across years to determine the optimal time period for analysis.

In [ ]:
# Extract column names (metric names) for the combined DataFrame
Col_names=[]
for i in range(0,len(missing_list)):
    Col_names.append(missing_list[i].index[0])

print(Col_names)

# Create combined DataFrame with missing data percentages by year
df_miss_year=pd.concat(missing_list,axis=1,keys= [f"{i}" for i in Col_names])
#Sample of missing data by year
print(df_miss_year.head())

### Statistical Summary of Missing Data

In [ ]:
print(df_miss_year.describe())
print(df_miss_year.info())

### Handling Missing Years in Analysis
Fill NaN values with 100% to represent complete absence of data for those years.

In [ ]:
df_miss_year.fillna(100,inplace=True)
df_miss_year.describe()

### Preparing Data for Time Period Visualization

Filter and prepare data for plotting to identify optimal time periods.

In [ ]:
# df_miss_year.fillna(100,inplace=True)
df_miss_year.index = df_miss_year.index.astype(str)

# Filter rows where index represents a valid 4-digit year
valid_years = df_miss_year.index[df_miss_year.index.str.match(r'^\d{4}$')]
df_m_plot = df_miss_year.loc[valid_years].copy()
df_m_plot.index = df_m_plot.index.astype(int)

### Visualizing Missing Data Trends by Year

Create a comprehensive visualization to identify the optimal time period for analysis.

In [ ]:
# Calculate average missing data percentage per year
average_missing_per_year = df_m_plot.mean(axis=1)
average_missing_per_year = average_missing_per_year.sort_index()

# Create decade markers for x-axis
decade_years = [year for year in average_missing_per_year.index if year % 10 == 0]
# Define target period and calculate its average
y_value=average_missing_per_year.loc[1990:2023].mean()

# Create the visualization
plt.figure(figsize=(12, 6))
plt.bar(average_missing_per_year.index, average_missing_per_year.values, color='mediumseagreen')

# Highlight target period
plt.plot([1990,2023],[y_value,y_value],color='red', linestyle='--', linewidth=2, label=f'Average from 1990-2023 ({y_value:.2f}%)')
plt.xlabel('Year')
plt.ylabel('Average Missing Data Percentage')
plt.title('Average Missing Data Percentage by Year')
plt.xticks(decade_years, rotation=45)
plt.legend()
plt.tight_layout()
plt.show()

### Time Period Selection Conclusion

**Selected Period: 1990-2023**
The analysis shows that the period from 1990 to 2023 provides an optimal balance between:
- **Data availability**: Average missing data percentage is 17.39%
- **Time span**: 34 years of data for robust analysis
- **Data quality**: Below the 20% threshold for acceptable missing data.

This period will be used for all subsequent analysis.

<a id="country-analysis"></a>
## 4. Country Selection Analysis
### Preparing Data for Country-Level Analysis
Extract data for the selected time period and prepare for country-level missing data analysis.

In [ ]:
# Extract data for 1990-2023 period
list_1990=[]

for i, df in enumerate(df_list,1):
    # Extract country column (first column, excluding header rows)
    df_i=df.iloc[1:-2,0] # Skip header and footer rows
    # Extract data for years 1990-2023
    df_col= df.iloc[1:-2,:].loc[:,1990:2023]
    # Combine country names with year data
    df_1990=pd.concat([df_i,df_col],axis=1,ignore_index=False)
    list_1990.append(df_1990)
    print(f"   Shape: {df_1990.shape}")
    print(f"   Sample countries: {df_1990.iloc[:3, 0].tolist()}")


### Analyzing Missing Data by Country
Find the missing percentage for each country on each metric. **Note** that NaN translates as the country is not included in the table.

In [ ]:
# Calculate missing data percentages by country
miss_1990=[]

for df_m in list_1990:
    # Transpose to have countries as columns
    df_m=df_m.T
    # Set first row as column names (country names)
    df_m.columns = df_m.iloc[0]
    # Calculate missing data percentage for each country
    miss_1990.append(df_m.isnull().mean()*100)

df_miss_country=pd.concat(miss_1990,axis=1,keys=Col_names)
# Sample of missing data by country
print(df_miss_country.head())


### Visualizing Country Data Quality Distribution
Create visualizations to understand the distribution of data quality across countries. We fill the nan values with 100 to map as 100% missing data. Make a histogram to visualise the missing data per country

In [ ]:
# Calculate average missing data percentage across all metrics for each country
avg_miss = df_miss_country.fillna(100).mean(axis=1)

# Overall distribution histogram
plt.hist(avg_miss)
plt.ylabel('Count')
print('Total number of Countries: ' + str(avg_miss.count()))

We see that there are a lot of countries above 50 % but a sizeble number still remane under 40%. Let's Focus on countries with less than 40% missing data for detailed analysis.

In [ ]:
# Analyze countries with <40% missing data
miss_40 = avg_miss[avg_miss < 40]
print('Number of Counries with average missing persentage below 40: '+ str(miss_40.count()))
plt.figure(figsize=(10, 5))


miss_40.plot(kind='hist', color='red', alpha=0.7)
plt.title('Countries with Average Percentage Below 40%')
plt.ylabel('Count')
plt.show()

**(Optional)** Find the countries that have less than 40% average missing data in all metrics. This criteria results in a more restricted selection than we choose later. It's commented out so we can select it later if interested.

In [ ]:
"""
filtered_list=[]
for i,df in enumerate(list_1990):
    name=Col_names[i]
    df = df.set_index(df.columns[0])
    common_indexes = df.index.intersection(miss_40.index)
    filtered_df = df.loc[common_indexes]
    filtered_df.columns.name = str(name)
    filtered_list.append(filtered_df)
    print(filtered_df.head)
"""


### Final Country Selection: Balanced Approach


**Goal**: Find coutries that have more than 70% in all metrics.

**Selection Criteria**: Countries with ≤30% missing data in ANY metric.
This approach:
- Removes countries with >30% missing data in any single metric
- Balances data quality with country coverage

**Note**: We don't fill NaN with 100, because NaN translates as the country not exicting in the table so filling with 100 will result in removing it entierly from the list and missing infromation.


In [ ]:
# Filter countries: remove those with >30% missing data in ANY metric
df_miss_30 = df_miss_country[~(df_miss_country >30).any(axis=1)]

# Countries meeting criteria
print('Number of selected countries: ' + str(df_miss_30.shape[0]))

filtered_list=[]
for i,df in enumerate(list_1990):
    name=Col_names[i]
    df = df.set_index(df.columns[0])
    common_indexes = df.index.intersection(df_miss_30.index)
    filtered_df = df.loc[common_indexes]
    filtered_df.columns.name = str(name)
    filtered_list.append(filtered_df)



### Cross-Dataset Country Coverage Analysis
Identify countries that exicts in ALL datasets after filtering. If we have chosen different criteria their might be more countries across all tables that might allow for a group, so we can make deeper analysis. 

In [ ]:
# Start with countries from first dataset
common_ind = set(filtered_list[0].index)

# Find intersection with all other datasets
for df in filtered_list[1:]:
    common_ind.intersection_update(df.index)

print("Common indexes:", common_ind)

<a id="data-imputation"></a>
## 5. Data Imputation and Final Cleaning
### Missing Data Imputation Strategy
**Approach**: Linear interpolation with constraints
- **Method**: Linear interpolation for temporal data
- **Direction**: Both forward and backward (leading and trailing NaNs)
- **Limit**: Maximum 5 consecutive missing values
- **Final step**: Drop any remaining rows with missing values

This ensures we maintain data quality while filling reasonable gaps in time series data.

In [ ]:
# Apply interpolation to fill missing values
filled_list=[]
for i,df in enumerate(filtered_list):
    # Count missing values before imputation
    missing_counts = df.isnull().sum().sum()
    print(f'Table {i+1}: Missing Before  '+str(missing_counts))
    df_fill=df.interpolate(
        method='linear', # Linear interpolation
        limit_direction='both', # Fill both leading and trailing NaNs
        limit=5, # Maximum 5 consecutive missing values\n
        axis=1 # Interpolate along columns (years)
        )  
    # Count missing values after imputation
    missing_counts = df_fill.isnull().sum().sum()
    print(f'Table {i+1}: Missing After   '+str(missing_counts)+'\n')
    # Drop rows that still have missing values
    df_fill.dropna(inplace=True)
    filled_list.append(df_fill)


<a id="bis-integration"></a>
## 6. BIS Data Integration
### Bank for International Settlements (BIS) Data Processing
The BIS data has a different structure and requires separate processing:
- **Metrics**: Debt service ratios for households and corporations
- **Period**: 1999-2023 (different from main dataset) and orginally quarterly. 
- **Sheet**: Data is in sheet 2 of the Excel files after averaging annually the original quarterly data.

In [ ]:
bis_list=[]
for item in ignore_list:
    # Extract filename for labeling
    df_name=item.split('\\')[-1]
    df_name=df_name.replace('.xlsx', '') 
    df_bis = pd.read_excel(item, engine="openpyxl",sheet_name=2)
    # Set country names as index
    df_bis = df_bis.set_index(df_bis.columns[0])
    # Rename year columns from string to integer
    df_bis.columns.name = str(df_name)
    for i in range(1999,2024):
        df_bis.rename(columns={f'{i}':i},inplace=True)

    # Remove 2024 column
    df_bis.drop(columns=['2024'],inplace=True)
    # Sort columns by year
    df_bis.sort_index(axis=1, inplace=True)
    df_bis.index.name = None
    bis_list.append(df_bis)


### Combining All Datasets

In [ ]:
data_list= filled_list + bis_list

<a id="data-format"></a>
## 7. Data Format Transformation
### Converting to Long Format
Transform the wide-format data (countries × years) into long format for easier analysis and storage:

In [ ]:
# Create metric names list (combining main and BIS datasets)
names =Col_names+ ['Debt service ratios - Households (BIS)', 'Debt service ratios - Nonfinancial corporate (BIS)']  # Replace with your actual names

# Combine all datasets with metric identifiers
# Create combined DataFrame with hierarchical index
combined_df = pd.concat(data_list, keys=names, names=['Metric', 'Country'])

# Convert to long format
long_df = combined_df.reset_index().melt(id_vars=['Metric', 'Country'], 
                           var_name='Year', 
                           value_name='Value')

In [ ]:
combined_df.head()

### Creating User-Friendly Variable Names

In [ ]:
# Define mapping from long names to short, analysis-friendly names
column_mapping = {
    'GDP, current prices (Billions of U.S. dollars)': 'GDP_USD_bn',
    'General Government Debt (Percent of GDP)': 'Govt_Debt_GDP',
    'Household debt, all instruments (Percent of GDP)': 'Household_Debt_GDP',
    'Inflation rate, average consumer prices (Annual percent change)': 'Inflation',
    'Interest paid on public debt, percent of GDP (% of GDP)': 'Interest_Paid_GDP',
    'Nonfinancial Public Sector Debt (Percent of GDP)': 'Public_Sector_Debt_GDP',
    'Nonfinancial corporate debt, all instruments (Percent of GDP)': 'Corporate_Debt_GDP',
    'Population (Millions of people)': 'Population_mn',
    'Unemployment rate (Percent)': 'Unemployment',
    'Debt service ratios - Households (BIS)': 'Debt_Service_Households',
    'Debt service ratios - Nonfinancial corporate (BIS)': 'Debt_Service_Corporates'
}

long_df['Metric'] = long_df['Metric'].replace(column_mapping)

long_df.head()

### Dataset Summary and Export
Create a final summary of the processed dataset and export it for analysis.

In [ ]:
print(f"   Total observations: {len(long_df)}")
print(f"   Unique countries: {long_df['Country'].nunique()}")
print(f"   Unique metrics: {long_df['Metric'].nunique()}")
print(f"   Time period: {long_df['Year'].min()} - {long_df['Year'].max()}")      


In [ ]:
long_df.to_csv('Processed_Data.csv',index=False)